# Look for coevolution Streptococcus-Lactobacillus

Coevolution is everywhere. Because no species can be considered totally isolated from the others. And not only on a biological level, we perceive it in the lines of change in our societies, in political, scientific, religious ideas and even in the evolution of software engineering.

The difficulty is to know how much co-evolution is due to the interaction between a pair of species, because the tandem can not be totally isolated from the rest of the universe either. No doubt the problem can not be solved only from the bioinformatic perspective, it is also necessary the contribution of other branches of knowledge such as microbiology, physics, mathematics and computer theory.

Our approach consists in comparing the rates of protein evolutionary change between strains of Streptococcus and Lactobacillus linked by symbiotic pathways, and the rates between the rest of species of each genus.

We postulate that the proteins that reflects a rate change with more significance, i.e., more speedy or more slowly that intra genus rates, are proteins that could be influenced by the symbiotic environment.

To do so we need to compute separately both species of symbiotic tandem. The detailed steps are:

1) Obtain all the proteome from this four groups of species:
- Genus streptococcus
- Strains of streptococcus termophilus.
- Genus lactobacillus
- Strains of lactobacillus bulgaricus.

2) For each protein of each group we obtain the phylogenetic tree and compute the mean branch length. We suppose that we have ultrametricicy or almost ultrametricity. It can be not exact but it could serve as a reference. We use the clulstalw multialignment method, but other methods as T-COFFEE or MUSCLE could be used too. The branch length is a measure of the substitution rate.

3) For each protein of the first an second group, we calculate the ration between branch length of termophilus and branch length of protein in genus. And also we compute the overall mean of all ratios. Finally we select the proteins with the most extreme values (80% percentile, two tailed).

4) We do the same with lactobacillus (third an fourth groups). At this stage we have two sets of proteins.

5) We obtain the biologic pathways of each of the sets of proteins.

6) The proteins involved in the same pathways from lactobacillus ans streptoccocus are the target proteins influenced by the coevolution between the two species. It will be necessary a later microbiological study that confirms this expectancies and dive deeply in the details of this coevolution.


## Obtain data from servers

### Methods

In [109]:
import requests, sys
import pickle
import numpy as np

# Disc serialization methods
def dump(obj, obj_name):
    binary_file = open(obj_name + '.bin',mode='wb')
    pickled_obj = pickle.dump(obj, binary_file)
    binary_file.close()

def load(obj_name):
    with (open(obj_name + ".bin", "rb")) as openfile:
        while True:
            try:
                obj = pickle.load(openfile)
            except EOFError:
                break
    return obj

def load_taxa(scientific_prefix):
    """
    Inputs:
        - scientific_prefix: string of the species
    Return:
        - taxa: list of taxonomyIds
        - names: list of scientificNames
    """
    
    requestURL = "https://www.ebi.ac.uk/proteins/api/taxonomy/name/" + scientific_prefix +\
                "%20?pageNumber=1&pageSize=100&searchType=STARTSWITH&fieldName=SCIENTIFICNAME"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    jsonBody = json.loads(r.text)
    taxa = []
    names = []
    for taxonomy in jsonBody["taxonomies"]:
        print(taxonomy['taxonomyId'], taxonomy['scientificName'])
        taxa.append(taxonomy['taxonomyId'])
        names.append(taxonomy['scientificName'])
    return taxa, names

def load_proteome(taxids, size=10, protein=["LDH"]):
    """
    Inputs:
        - taxids: list of taxonomyIds
        - size: integer that indicates the number of sequences
                we want to explore (if it's equal to -1, we will
                obtain all of the sequences)
        - protein: list of proteins
    Return:
        - proteome: proteome
    """
    taxids_str = ",".join(str(x) for x in taxids)
    protein_str = ",".join(x for x in protein)
    print(taxids_str)
    requestURL = "https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=" + str(size) + "&taxid=" +\
                    taxids_str + "&reviewed=false"
    if protein != []:
        requestURL += "&gene=" + protein_str 
    print(requestURL)
    r = requests.get(requestURL, headers={ "Accept" : "text/x-fasta"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    proteome = r.text
    return proteome

### Load streptococcus taxids

In [112]:
termophilus_taxa,  termophilus_names = load_taxa("Streptococcus thermophilus")
streptococcus_taxa,  streptococcus_names = load_taxa("Streptococcus")
dump(termophilus_taxa, "termophilus_taxa")
dump(termophilus_names, "termophilus_names")
dump(streptococcus_taxa, "streptococcus_taxa")
dump(streptococcus_names, "streptococcus_names")

264199 Streptococcus thermophilus (strain ATCC BAA-250 / LMG 18311)
299768 Streptococcus thermophilus (strain CNRZ 1066)
322159 Streptococcus thermophilus (strain ATCC BAA-491 / LMD-9)
767463 Streptococcus thermophilus (strain ND03)
1042404 Streptococcus thermophilus CNCM I-1630
1051074 Streptococcus thermophilus JIM 8232
1073569 Streptococcus thermophilus MTCC 5460
1073570 Streptococcus thermophilus MTCC 5461
1091038 Streptococcus thermophilus DSM 20617
1187956 Streptococcus thermophilus MN-ZLW-002
1263110 Streptococcus thermophilus CAG:236
1268061 Streptococcus thermophilus DGCC 7710
1408178 Streptococcus thermophilus ASCC 1275
1415776 Streptococcus thermophilus TH1435
1423145 Streptococcus thermophilus TH1436
1433288 Streptococcus thermophilus MTH17CL396
1433289 Streptococcus thermophilus M17PTZA496
1435972 Streptococcus thermophilus TH985
1435974 Streptococcus thermophilus TH982
1435981 Streptococcus thermophilus 1F8CT
1436725 Streptococcus thermophilus TH1477
1302 Streptococcus go

### Load lactobacillus taxids

In [113]:
lactobacillus_taxa,  lactobacillus_names = load_taxa("Lactobacillus")
bulgaricus_taxa,  bulgaricus_names = load_taxa("Lactobacillus delbrueckii subsp. bulgaricus")

dump(lactobacillus_taxa, "lactobacillus_taxa")
dump(lactobacillus_names, "lactobacillus_names")
dump(bulgaricus_taxa, "bulgaricus_taxa")
dump(bulgaricus_names, "bulgaricus_names")

1579 Lactobacillus acidophilus
1580 Lactobacillus brevis
1581 Lactobacillus buchneri
1582 Lactobacillus casei
1584 Lactobacillus delbrueckii
1585 Lactobacillus delbrueckii subsp. bulgaricus
1587 Lactobacillus helveticus
1588 Lactobacillus hilgardii
1589 Lactobacillus pentosus
1590 Lactobacillus plantarum
1591 Lactobacillus sp.
1593 Lactobacillus sp. (strain 30a)
1596 Lactobacillus gasseri
1597 Lactobacillus paracasei
1598 Lactobacillus reuteri
1599 Lactobacillus sakei
1600 Lactobacillus acetotolerans
1601 Lactobacillus agilis
1602 Lactobacillus alimentarius
1603 Lactobacillus amylophilus
1604 Lactobacillus amylovorus
1605 Lactobacillus animalis
1606 Lactobacillus aviarius
1607 Lactobacillus bifermentans
1610 Lactobacillus coryniformis
1612 Lactobacillus farciminis
1613 Lactobacillus fermentum
1614 Lactobacillus fructivorans
1618 Lactobacillus mali
1622 Lactobacillus murinus
1623 Lactobacillus ruminis
1624 Lactobacillus salivarius
1625 Lactobacillus sanfranciscensis
1626 Lactobacillus s

### Load streptococcus proteomes

In [41]:
RESTART = True
VERBOSE = True
# If [] return all the proteins
PROTEIN_LIST = []

if RESTART:
    termophilus_taxa = load("termophilus_taxa")
    streptococcus_taxa = load("streptococcus_taxa")
    
# We limit to 20 taxids (it's the maximum for webservice and it should be enough
termophilus_taxids = termophilus_taxa[0:19]
streptococcus_taxids = streptococcus_taxa[0:19]
print(streptococcus_taxids)
print(termophilus_taxids)

#streptococcus_proteome = load_proteome(streptococcus_taxids, -1, protein = ["LDH", "CAS2", "CAS3"])
#termophilus_proteome = load_proteome(termophilus_taxids, -1, protein = ["LDH", "CAS2", "CAS3"])

streptococcus_proteome_complete = load_proteome(streptococcus_taxids, -1, PROTEIN_LIST)
termophilus_proteome_complete = load_proteome(termophilus_taxids, -1, PROTEIN_LIST)

dump(streptococcus_proteome_complete, "streptococcus_proteome_complete")
dump(termophilus_proteome_complete, "termophilus_proteome_complete")

[1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1313, 1314, 1317, 1318, 1319, 1320, 1324, 1325, 1326]
[264199, 299768, 322159, 767463, 1042404, 1051074, 1073569, 1073570, 1091038, 1187956, 1263110, 1268061, 1408178, 1415776, 1423145, 1433288, 1433289, 1435972, 1435974]
1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1313,1314,1317,1318,1319,1320,1324,1325,1326
https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=-1&taxid=1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1313,1314,1317,1318,1319,1320,1324,1325,1326&reviewed=false
264199,299768,322159,767463,1042404,1051074,1073569,1073570,1091038,1187956,1263110,1268061,1408178,1415776,1423145,1433288,1433289,1435972,1435974
https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=-1&taxid=264199,299768,322159,767463,1042404,1051074,1073569,1073570,1091038,1187956,1263110,1268061,1408178,1415776,1423145,1433288,1433289,1435972,1435974&reviewed=false


In [63]:
print(termophilus_proteome_complete[0:1000])
print(streptococcus_proteome_complete[0:1000])

>tr|V6CG59|V6CG59_STRTN Proteolysis tag peptide encoded by tmRNA Strep_therm_CNRZ10 (Fragment) OS=Streptococcus thermophilus (strain ND03) OX=767463 GN=tmRNA Strep_therm_CNRZ10 PE=4 SV=1
AKNTNSYAVAA
>tr|V6CG54|V6CG54_STRTR Proteolysis tag peptide encoded by tmRNA Strep_therm (Fragment) OS=Streptococcus thermophilus MN-ZLW-002 OX=1187956 GN=tmRNA Strep_therm PE=4 SV=1
AKNTNSYAVAA
>tr|V6CG50|V6CG50_STRTR Proteolysis tag peptide encoded by tmRNA Strep_therm_CNRZ10 (Fragment) OS=Streptococcus thermophilus MTCC 5461 OX=1073570 GN=tmRNA Strep_therm_CNRZ10 PE=4 SV=1
AKNTNSYAVAA
>tr|V6BJW4|V6BJW4_STRTR Proteolysis tag peptide encoded by tmRNA Strep_therm_CNRZ10 (Fragment) OS=Streptococcus thermophilus MTCC 5460 OX=1073569 GN=tmRNA Strep_therm_CNRZ10 PE=4 SV=1
AKNTNSYAVAA
>tr|V6CE29|V6CE29_STRT1 Proteolysis tag peptide encoded by tmRNA Strep_therm_CNRZ10 (Fragment) OS=Streptococcus thermophilus (strain CNRZ 1066) OX=299768 GN=tmRNA Strep_therm_CNRZ10 PE=4 SV=1
AKNTNSYAVAA
>tr|A0A1L1QK15|A0A1L1Q

### Load lactobacillus proteomes

In [114]:
RESTART = True
VERBOSE = True
# If [] return all the proteins
PROTEIN_LIST = []

if RESTART:
    lactobacillus_taxa = load("lactobacillus_taxa")
    lactobacillus_names = load("lactobacillus_names")
    bulgaricus_taxa = load("bulgaricus_taxa")
    bulgaricus_names = load("bulgaricus_names")
# We limit to 20 taxids (it's the maximum for webservice and it should be enough
lactobacillus_taxids = lactobacillus_taxa[0:19]
bulgaricus_taxids = bulgaricus_taxa[0:19]
print(lactobacillus_taxids)
print(bulgaricus_taxids)

lactobacillus_proteome_complete = load_proteome(lactobacillus_taxids, -1, PROTEIN_LIST)
bulgaricus_proteome_complete = load_proteome(bulgaricus_taxids, -1, PROTEIN_LIST)

dump(lactobacillus_proteome_complete, "lactobacillus_proteome_complete")
dump(bulgaricus_proteome_complete, "bulgaricus_proteome_complete")

[1579, 1580, 1581, 1582, 1584, 1585, 1587, 1588, 1589, 1590, 1591, 1593, 1596, 1597, 1598, 1599, 1600, 1601, 1602]
[321956, 353496, 390333, 767455, 784613, 1042399, 1042400]
1579,1580,1581,1582,1584,1585,1587,1588,1589,1590,1591,1593,1596,1597,1598,1599,1600,1601,1602
https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=-1&taxid=1579,1580,1581,1582,1584,1585,1587,1588,1589,1590,1591,1593,1596,1597,1598,1599,1600,1601,1602&reviewed=false
321956,353496,390333,767455,784613,1042399,1042400
https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=-1&taxid=321956,353496,390333,767455,784613,1042399,1042400&reviewed=false


## Compute substitution rates

### Methods

In [52]:
import re

def proteome2dict(proteome_fasta):
    """
    Returns a dict with keys protein accession and values the list of fasta format for all taxids
    This is the basis for clustalw alignments and tree generation
    """
    proteome = {}
    key_found = False
    for line in proteome_fasta.splitlines():
        if len(line) > 0:
            if line[0] == ">":
                if key_found:
                    if key in proteome:
                        proteome[key].append(seq)
                    else:
                        proteome[key] = [seq]  
                search_gene_name = re.search('GN=(\w*)', line)
                if search_gene_name:
                    key = search_gene_name.group(1).upper()
                    key_found = True  
                #print(key)
                seq = line + '\n'
            elif key_found:
                seq += line + '\n'
    if key_found:
        if key in proteome:
            proteome[key].append(seq)
        else:
            proteome[key] = [seq]
    return proteome

In [80]:
# Phylo tree with clustalw. We need to measure the substitution rate.
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from Bio import Phylo
from io import StringIO
import os
from Bio.Align.Applications import ClustalwCommandline
CLUSTALW = r"./clustalw2"
assert os.path.isfile(CLUSTALW), "Clustal W executable missing"
plt.rcParams["figure.figsize"] = (20,30)
matplotlib.rc('font', size=12)
        
def compute_mean_subst_rate(proteome, verbose=False, show_tree=False):
    """
    Function that implements the basis for ClustalW alignments and tree generation.
    """
    clustalw_cline = ClustalwCommandline(CLUSTALW, infile=proteome + ".fasta")
    stdout, stderr = clustalw_cline()
    f = open(proteome + ".dnd", "r")
    s_tree = f.read()
    f.close()
    #print(s_tree)
    branch_len = 0
    num_branches = 0
    search_branch_length = re.findall(':([-.0123456789]*)', s_tree)
    for branch_length in search_branch_length:
        #print(branch_length)
        if branch_length != "0.00000":
            branch_len += float(branch_length)
            num_branches += 1
    if num_branches > 0:
        rate = branch_len/num_branches
    else:
        rate = -1
    if verbose: print(branch_len, num_branches, rate)
    if show_tree:
        tree = Phylo.read(proteome + ".dnd", "newick")
        Phylo.draw(tree)
    return rate

def compute_subst_rates(proteome, proteome_keys, proteome_name, verbose=False):
    """
    """
    subst_rates = {}
    for protein in proteome_keys:
        if verbose: print(protein)
        protein_sequence = ""
        # Only for proteins with enough sequences to make a tree
        if len(proteome[protein]) >= 3:
            for sequence in proteome[protein]:
                protein_sequence += sequence
            fasta_file_name = proteome_name + "_" + protein
            f = open(fasta_file_name + ".fasta", "w")
            if verbose: print(protein_sequence)
            f.write(protein_sequence)
            f.close()
            mean_subst_rate = compute_mean_subst_rate(fasta_file_name)
            subst_rates[protein] = mean_subst_rate
    return subst_rates

def select_proteins_v1(branches_length, genus, strains):
    # We save the proteins presents in the both groups
    proteins = list(set(branches_length[genus].keys()) & set(branches_length[strains].keys()))
    print('proteins:', proteins)
    
    # Compute branch ratios
    ratios = {}
    for p in proteins:
        print(branches_length[strains][p], branches_length[genus][p])
        ratios[p] = branches_length[strains][p]/branches_length[genus][p]
    list_ratios = np.array(list(ratios.values()))
    print('list_ratios:', list_ratios)
    
    # Compute mean of branch ratios and standard deviation
    mean_ratios = list_ratios.mean()
    sd_ratios = list_ratios.std()
    
    # Obtain the most extreme values.
    ## These are the proteins that could have been a slowdown or from his initial state.
    extreme_values = np.percentile(list_ratios, 80), np.percentile(list_ratios, 20)

    return extreme_values

def select_proteins_v2(branches_length, genus, strains):
    # We divided proteins of the same type to calculate coherent ratios
    proteins = {}
    for ps in branches_length[strains]:
        proteins[ps] = []
        for pg in branches_length[genus]:
            if pg.startswith(ps):
                proteins[ps].append(pg)
    print('proteins:', proteins)

### Fasta to dictionary

In [64]:
#Proteomes in fasta to dictionaries
RESTART = True
VERBOSE = True

if RESTART:
    termophilus_proteome_fasta = load("termophilus_proteome_complete")
    streptococcus_proteome_fasta = load("streptococcus_proteome_complete")

#print(termophilus_proteome_fasta)
proteome_termophilus = proteome2dict(termophilus_proteome_fasta)
proteome_streptococcus = proteome2dict(streptococcus_proteome_fasta)

dump(proteome_termophilus, "proteome_termophilus")
dump(proteome_streptococcus, "proteome_streptococcus")

if VERBOSE: print(list(proteome_termophilus.keys())[0:100])
if VERBOSE: print(list(proteome_streptococcus.keys())[0:100])

['TMRNA', 'BN551_00358', 'GALT', 'LACZ', 'SBCD', 'GALR', 'DNAN', 'STU0007', 'FTSH', 'MREC', 'ARAT', 'PURL', 'STU0044', 'STU0052', 'STU0053', 'TAG', 'STU0075', 'STU0082', 'LABC', 'STU0110', 'STU0113', 'ILVA', 'STU0161', 'STU0182', 'STU0202', 'STU0208', 'STU0251', 'STU0258', 'STU0267', 'CBIM', 'STU0297', 'TRKH2', 'DNAB', 'SERS', 'STU0330', 'STU0334', 'STU0338', 'STU0339', 'METB1', 'STU0358', 'LIVM', 'HPF', 'FABF', 'FRUR', 'STU0422', 'STU0435', 'PHOH', 'STU0448', 'METS', 'STU0452', 'PRTM', 'ARGB', 'STU0468', 'STU0473', 'STU0474', 'STU0475', 'FTSW', 'DNAH', 'NAGA', 'STU0510', 'STU0516', 'STU0539', 'STU0551', 'STU0557', 'STU0565', 'HLYIII', 'STU0580', 'STU0595', 'MURN', 'NNRD', 'RNR', 'STU0631', 'STU0636', 'AROE', 'MIP', 'CAS2', 'STU0665', 'STU0668', 'STU0672', 'STU0675', 'STU0678', 'STU0679', 'STU0681', 'STU0693', 'STU0695', 'GPMC', 'STU0702', 'STU0704', 'LEMA', 'STU0721', 'APBE', 'DLTX', 'STU0811', 'STU0819', 'STU0829', 'ADCA', 'MUR2', 'STU0876', 'STU0877', 'STHIM']
['SRTA', 'ATLH', 'DBLB

In [115]:
#Proteomes in fasta to dictionaries
RESTART = True
VERBOSE = True

if RESTART:
    bulgaricus_proteome_fasta = load("bulgaricus_proteome_complete")
    lactobacillus_proteome_fasta = load("lactobacillus_proteome_complete")

proteome_bulgaricus = proteome2dict(bulgaricus_proteome_fasta)
proteome_lactobacillus = proteome2dict(lactobacillus_proteome_fasta)

dump(proteome_bulgaricus, "proteome_bulgaricus")
dump(proteome_lactobacillus, "proteome_lactobacillus")

if VERBOSE: print(list(proteome_bulgaricus.keys())[0:100])
if VERBOSE: print(list(proteome_lactobacillus.keys())[0:100])

['TMRNA', 'EXOA', 'LDB0010', 'LDB0021', 'LDB0026', 'LDB0031', 'LDB0191', 'LDB0200', 'PHNB', 'LDB0206', 'LDB0207', 'NRDG', 'LDB0226', 'LDB0229', 'LDB0233', 'LDB0252', 'LDB0271', 'OPPA3II', 'OPPCII', 'OPPDII', 'LDB0301', 'LDB0313', 'LDB0314', 'LDB0327', 'LDB0329', 'LDB0332', 'LDB0351', 'CSHA', 'LDB0365', 'FTSH', 'LYSS', 'LDB0374', 'LDB0378', 'CBIQ', 'LDB0431', 'LDB0432', 'LDB0435', 'LDB0438', 'LDB0447', 'LDB0448', 'LDB0453', 'NADE', 'LDB0482', 'PEPD1', 'UDK1', 'GLNH1', 'GLNP', 'PPX', 'LDB0533', 'LDB0541', 'FLAV', 'LDB0563', 'LDB0567', 'PTSH', 'PTSI', 'SPX', 'HPF', 'UVRB', 'UVRA', 'LDB0631', 'CGGR', 'GAP', 'POTD1', 'LDB0671', 'LDB0675', 'COMGC', 'ACK', 'LDB0689', 'LDB0691', 'HEMK', 'MREB2', 'FTSA', 'FTSZ', 'LDB0754', 'RECD', 'RNJ', 'LDB0808', 'LDB0812', 'LDB0827', 'LDB0844', 'LDB0849', 'LDB0861', 'LDB0877', 'FABF', 'ACCC', 'LDB0930', 'PHOU', 'LDB0967', 'LDB0977', 'LDB0980', 'MVAD', 'DNAD', 'NTH', 'LDB1015', 'LDB1019', 'LSP', 'LDB1025', 'LDB1031', 'LDB1099', 'LDB1087']
['TETM', 'RPOA', 'AT

### Substitution rates

In [132]:
#Proteomes in fasta to dictionaries
RESTART = True
VERBOSE = True
#Protein limit
LIMIT = 200 

if RESTART:
    proteome_termophilus = load("proteome_termophilus")
    proteome_streptococcus = load("proteome_streptococcus")

# Compute keys to process: only the keys that are included in both groups
proteome_keys = []
for key in proteome_termophilus.keys():
    if key in proteome_streptococcus.keys():
        proteome_keys.append(key)

if VERBOSE: print(proteome_keys[0:LIMIT])
print("Proteins we need to process:", len(proteome_keys))
limit =  min(len(proteome_keys), LIMIT) 
print("Proteins we want to process:", limit)

# Compute branch lengths
subst_rates_groups = {}
subst_rates_groups["termophilus"] = compute_subst_rates(proteome_termophilus, proteome_keys[0:limit],
                                                        "termophilus", False)
subst_rates_groups["streptococcus"] = compute_subst_rates(proteome_streptococcus, proteome_keys[0:limit], 
                                                          "streptococcus", False)

dump(subst_rates_groups, "subst_rates_groups")
if VERBOSE: print(subst_rates_groups)  

['TMRNA', 'GALT', 'LACZ', 'SBCD', 'GALR', 'DNAN', 'FTSH', 'MREC', 'ARAT', 'PURL', 'TAG', 'LABC', 'ILVA', 'CBIM', 'TRKH2', 'DNAB', 'SERS', 'LIVM', 'HPF', 'FABF', 'FRUR', 'PHOH', 'METS', 'PRTM', 'ARGB', 'FTSW', 'DNAH', 'NAGA', 'HLYIII', 'MURN', 'NNRD', 'RNR', 'AROE', 'MIP', 'CAS2', 'GPMC', 'LEMA', 'APBE', 'DLTX', 'ADCA', 'MUR2', 'STHIM', 'HK04', 'SPEB', 'FBP', 'DTPT', 'PCRA', 'RIBC', 'PSTC2', 'TATA', 'TNP1191', 'ACOL', 'EPS3', 'FER', 'SIPB', 'ESTA', 'PFK', 'RECN', 'ARGR', 'DNAD', 'HFLX', 'CITB', 'ASD', 'CLS', 'PROWZ', 'HUTU', 'SDAB', 'HK06', 'RR07', 'NORM', 'SBCC', 'SUNL', 'FTSY', 'OXLT', 'PRSA2', 'REX', 'RGPF', 'RGPE', 'RGPD', 'RGPC', 'RGPA', 'RPOD', 'CORA1', 'CAH', 'GLA', 'BLPI', 'SCRB', 'MUTY', 'RECD', 'TRXA1', 'RADA', 'DUT', 'ACKA', 'COMGC', 'COMGB', 'INT', 'ARGS', 'PLSX', 'HRCA', 'METN', 'RECU', 'YIDC', 'TILS', 'THRB', 'PYRB', 'RSMI', 'MUTL', 'TMK', 'ATPB', 'PYRH', 'METG', 'FRR', 'PNP', 'RECX', 'RPOE', 'GRPE', 'AROA', 'TUF', 'RPSA', 'SSTT', 'GLMU', 'RPLU', 'HTPX', 'FTSZ', 'MURD', 'M

In [135]:
#Proteomes in fasta to dictionaries
RESTART = True
VERBOSE = True
#Protein limit
LIMIT = 200

if RESTART:
    proteome_bulgaricus = load("proteome_bulgaricus")
    proteome_lactobacillus = load("proteome_lactobacillus")

# Compute keys to process: only the keys that are included in both groups
proteome_keys = []
for key in proteome_bulgaricus.keys():
    if key in proteome_lactobacillus.keys():
        proteome_keys.append(key)

if VERBOSE: print(proteome_keys[0:LIMIT])
print("Proteins we need to process:", len(proteome_keys))
limit =  min(len(proteome_keys), LIMIT) 
print("Proteins we want to process:", limit)

# Compute branch lengths
subst_rates_groups_lacto = {}
subst_rates_groups_lacto["bulgaricus"] = compute_subst_rates(proteome_bulgaricus, proteome_keys[0:limit],
                                                        "bulgaricus", False)
subst_rates_groups_lacto["lactobacillus"] = compute_subst_rates(proteome_lactobacillus, proteome_keys[0:limit], 
                                                          "lactobacillus", False)

dump(subst_rates_groups_lacto, "subst_rates_groups_lacto")
if VERBOSE: print(subst_rates_groups_lacto)  

['TMRNA', 'EXOA', 'PHNB', 'NRDG', 'CSHA', 'FTSH', 'LYSS', 'CBIQ', 'NADE', 'PEPD1', 'GLNH1', 'GLNP', 'PPX', 'PTSH', 'PTSI', 'SPX', 'HPF', 'UVRB', 'UVRA', 'CGGR', 'GAP', 'COMGC', 'HEMK', 'FTSA', 'FTSZ', 'RECD', 'RNJ', 'FABF', 'ACCC', 'PHOU', 'MVAD', 'DNAD', 'NTH', 'LSP', 'MOD', 'POLC', 'OPPB', 'RECG', 'RPMA', 'PURF', 'PURS', 'CFA', 'PHET', 'PRSA', 'REX', 'PYRR2', 'PEPC', 'THIJ', 'THRC', 'GLPQ', 'GMK2', 'PARB1', 'PARA', 'DLTD', 'DLTB', 'DLTA', 'GLPF', 'DNAN', 'GYRB', 'MALG', 'PHND', 'PHNE', 'FOLP', 'GUAB', 'PURR', 'TRUA', 'PCRA', 'COPZ', 'GLMS', 'MURI', 'HPRT', 'PYRP', 'MAP', 'RGPG', 'PTA', 'COMGA', 'COMGD', 'EZRA', 'RSUA', 'MREC', 'HOLA', 'CAS1', 'MVAS', 'ACCB', 'FABI', 'PSTC', 'PARC', 'FNI', 'DING', 'PBP1A', 'PATC', 'LEPB', 'GLYQ', 'CLPQ', 'SRTA', 'CYSK2', 'SMC', 'XSEB', 'ADD', 'GLNA', 'GREA1', 'INFC', 'DNAI', 'PBP2A', 'FBA', 'MUTS2', 'GPD', 'SECE', 'PEPZ', 'PEPV', 'SECG', 'MPRF', 'GTCA', 'MSRB', 'RMLD', 'APBE', 'CITG', 'CLPL', 'PYRR1', 'CLS', 'ARBX', 'DNAC', 'NRDD', 'FOLB', 'PBUX', 'MS

## Selection of first set of target proteins
At this step is necessary to calculate the ratios, the means of the ratios and select the more extreme between them (two tailed percentile 80%).

### Methods

In [95]:
# Compute branch ratios
# Compute mean of branch ratios and standard deviation
# Obtain the most extreme values. 
# These are the proteins that could have been a slowdown or from his initial state
import statistics as stats

def compute_branch_ratios(subst_rates_groups, group1, group2, verbose=False):
    """ 
    For every protein in group1 that has counterpart in group2 calculate the ratio of
    branch lengths.
    
    Returns:
        dict of string, float: ratios by protein
    """
    ratios = {}
    for protein in subst_rates_groups[group1].keys():
        if protein in subst_rates_groups[group2].keys():
            ratio = subst_rates_groups[group1][protein]/(subst_rates_groups[group2][protein] + 0.00001)
            if subst_rates_groups[group1][protein] != -1 and subst_rates_groups[group2][protein] != -1:
                ratios[protein] = ratio
            if verbose: print(group1, protein, 
                              subst_rates_groups[group1][protein], 
                              subst_rates_groups[group2][protein],
                              ratio)
    return ratios

def compute_mean_std(ratios, verbose=False):
    """
    Calculate mean and std for ratios
    """
    ratios_list =  list(ratios.values())
    if verbose: print("ratios_list",ratios_list)
    ratios_mean = stats.mean(ratios_list)
    ratios_stdev = stats.stdev(ratios_list)
    return ratios_mean, ratios_stdev

def filter_target_proteins(ratios, mean, std, n_std, verbose=False):
    """
    Filter proteins that are n_std > mean or n_std < mean
    """
    proteins = []
    for protein in ratios.keys():
        if verbose: print(protein, ratios[protein], mean, std)
        if ratios[protein] > mean + n_std * std or ratios[protein] < mean - n_std * std:
            proteins.append(protein)
    return proteins

### Selection of target proteins

In [134]:
RESTART = True
VERBOSE = True

if RESTART:
    subst_rates_groups = load("subst_rates_groups")

rat = compute_branch_ratios(subst_rates_groups, "termophilus", "streptococcus", VERBOSE)   
if VERBOSE: print(rat)

if len(rat) > 1:
    mean, std = compute_mean_std(rat, VERBOSE)
    if VERBOSE: print("\nMean", mean, "Standard deviation", std, "\n")
    prot = filter_target_proteins(rat, mean, std, 0.8, VERBOSE)
    print("Target proteins Streptococcus:", prot)
else:
    print("No results: protein input number < 2")

termophilus TMRNA -1 0.10939888888888888 -9.14002518584718
termophilus GALT 0.014773333333333333 0.011721821086261982 1.2592532075547058
termophilus LACZ 0.00292 0.028357828947368432 0.10293350278646814
termophilus SBCD 0.00245625 0.05702195652173911 0.04306795961074456
termophilus FTSH 0.00262 0.017297005347593573 0.15138378635586971
termophilus ILVA 0.0024 0.012551562499999992 0.1910590342562879
termophilus SERS 0.001318888888888889 0.008190401785714287 0.16083222790211232
termophilus HPF 0.01282 0.03110622641509432 0.41200368672536347
termophilus FABF 0.31672 0.029105231788079465 10.878154853971433
termophilus NNRD 0.005995 0.01886595041322313 0.31759990192601567
termophilus RNR 0.00285 0.017501089494163423 0.16275400802159834
termophilus AROE 0.007016666666666667 0.02406172949002219 0.29148992678631996
termophilus CAS2 0.10117692307692308 0.05861141304347825 1.7259379776786055
termophilus PCRA 0.25916333333333336 0.01710847619047618 15.13939269182839
termophilus ARGR 0.089214444444

In [136]:
RESTART = True
VERBOSE = True

if RESTART:
    subst_rates_groups_lacto = load("subst_rates_groups_lacto")

rat = compute_branch_ratios(subst_rates_groups_lacto, "bulgaricus", "lactobacillus", VERBOSE)   
if VERBOSE: print(rat)

if len(rat) > 1:
    mean, std = compute_mean_std(rat, VERBOSE)
    if VERBOSE: print("\nMean", mean, "Standard deviation", std, "\n")
    prot = filter_target_proteins(rat, mean, std, 0.8, VERBOSE)
    print("Target proteins Lactobacillus:", prot)
else:
    print("No results: protein input number < 2")

bulgaricus TMRNA -1 0.09091 -10.998680158380994
bulgaricus CSHA 0.01663 0.029512741935483876 0.5632945624204412
bulgaricus FTSH 0.0023866666666666667 0.017883125000000003 0.13338456343800573
bulgaricus LYSS 0.01397 0.015703070175438597 0.8890687716673459
bulgaricus NADE 0.009055 0.020291111111111117 0.4460346997974932
bulgaricus HPF 0.00541 0.040429199999999985 0.13378108370096345
bulgaricus UVRB 0.0040425 0.0182108527131783 0.22186118639092267
bulgaricus UVRA 0.0035033333333333336 0.01776475 0.19709606792406834
bulgaricus FTSA 0.011955 0.030103584905660376 0.3969969048006918
bulgaricus FTSZ 0.0060825 0.029649259259259254 0.20507929570429576
bulgaricus RNJ 0.06184272727272726 0.018687327188940086 3.307570469714447
bulgaricus NTH 0.00478 0.03143635416666666 0.1520049025291088
bulgaricus POLC 0.0029900000000000005 0.01936439189189189 0.15432742440041716
bulgaricus RECG 0.0024533333333333334 0.01724738853503185 0.14216133155680877
bulgaricus RPMA 0.01003 0.06790682926829267 0.147680628027

## Final thoughts
At this stage we make a explanation of the results, justifications and propose a better computational approach.

It's not mandatory that we have confidence on this explanation, but it should seem credible.

# Generate document outputs.

In [35]:
%%bash
#cd /Users/nandoide/Desktop/uni/STRBI.practical
jupyter nbconvert --to=latex --template=~/report.tplx coevolution.ipynb 1> /dev/null
pdflatex -shell-escape coevolution 1> /dev/null
jupyter nbconvert --to html_with_toclenvs coevolution.ipynb 1> /dev/null

[NbConvertApp] Converting notebook coevolution.ipynb to latex
[NbConvertApp] Writing 93867 bytes to coevolution.tex
[NbConvertApp] Converting notebook coevolution.ipynb to html_with_toclenvs
[NbConvertApp] Writing 434566 bytes to coevolution.html
